In [ ]:
# Std libraries
import os
import time
import random
from google.colab import drive

# General libraries
%matplotlib inline
import PIL
import pathlib
import PIL.Image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# Tensorflow & Keras
import tensorflow as tf
from tensorflow.keras import models
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.initializers import RandomNormal, Constant
from tensorflow.keras.models import load_model, model_from_json, save_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

# Sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, roc_curve, auc, plot_confusion_matrix, classification_report, confusion_matrix, f1_score

In [ ]:
saved_model = load_model('foodclassification.hdf5', compile=False)

In [ ]:
model_json = saved_model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)

In [ ]:
from keras import model_from_json 

In [ ]:
json_file = open('model.json','r')
loaded_model_json = json_file.read()
json_file.close()

loaded_model = model_from_json(loaded_model_json)

loaded_model.load_weights('foodclassification.hdf5')
print("Loaded Model from disk")

loaded_model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

### Flask App

In [ ]:
from flask import Flask, render_template, request

In [ ]:
@app.route('/')
def index_view():
    return render_template('index.html')


@app.route('/predict/',methods=['GET','POST'])
def predict():
	response = "For ML Prediction"
  return response	

if __name__ == '__main__':
    app.run(debug=True, port=8000)

In [ ]:
from flask import Flask, render_template, request
from scipy.misc import imsave, imread, imresize
import numpy as np
import tensorflow.keras.models
import re
import base64
sys.path.append(os.path.abspath("./model"))
from load import * 

In [ ]:
global graph, model

model, graph = init()

app = Flask(__name__)

@app.route('/')
def index_view():
    return render_template('index.html')

def convertImage(imgData1):
	imgstr = re.search(b'base64,(.*)',imgData1).group(1)
	with open('output.png','wb') as output:
	    output.write(base64.b64decode(imgstr))

@app.route('/predict/',methods=['GET','POST'])
def predict():
	imgData = request.get_data()
	convertImage(imgData)
	x = imread('output.png',mode='L')
	x = np.invert(x)
	x = imresize(x,(28,28))
	x = x.reshape(1,28,28,1)

	with graph.as_default():
		out = model.predict(x)
		print(out)
		print(np.argmax(out,axis=1))

		response = np.array_str(np.argmax(out,axis=1))
		return response	

if __name__ == '__main__':
    app.run(debug=True, port=8000)